<a href="https://colab.research.google.com/github/fran1215/MLAI/blob/main/W5_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_03

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_03


# Task 1

## Pivoting / Unstack

### Pivoting

In [4]:
import pandas as pd

long_df = pd.read_csv(
    'data/long_data.csv', usecols=['date', 'datatype', 'value']
).rename(
    columns={'value': 'temp_C'}
).assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_F=lambda x: (x.temp_C * 9/5) + 32
)
long_df.head()

,datatype,date,temp_C,temp_F
0,TMAX,2018-10-01,21.1,69.98
1,TMIN,2018-10-01,8.9,48.02
2,TOBS,2018-10-01,13.9,57.02
3,TMAX,2018-10-02,23.9,75.02
4,TMIN,2018-10-02,13.9,57.02


In [5]:
pivoted_celsius = long_df.pivot(
    index='date',
    columns='datatype',
    values='temp_C'
)

In [9]:
pivoted_celsius.head()

datatype,TMAX,TMIN,TOBS
date,,,
2018-10-01,21.1,8.9,13.9
2018-10-02,23.9,13.9,17.2
2018-10-03,25.0,15.6,16.1
2018-10-04,22.8,11.7,11.7
2018-10-05,23.3,11.7,18.9


In [6]:
pivoted_fahrenheit = long_df.pivot(
    index='date',
    columns='datatype',
    values='temp_F'
)

In [10]:
pivoted_fahrenheit.head()

datatype,TMAX,TMIN,TOBS
date,,,
2018-10-01,69.98,48.02,57.02
2018-10-02,75.02,57.02,62.96
2018-10-03,77.00,60.08,60.98
2018-10-04,73.04,53.06,53.06
2018-10-05,73.94,53.06,66.02


### Unstack

In [15]:
multi_index_df = long_df.set_index(['date', 'datatype'])
multi_index_df.head()

temp_C  temp_F
date       datatype                
2018-10-01 TMAX        21.1   69.98
           TMIN         8.9   48.02
           TOBS        13.9   57.02
2018-10-02 TMAX        23.9   75.02
           TMIN        13.9   57.02

In [14]:
unstacked_df = multi_index_df.unstack()
unstacked_df.head()

temp_C             temp_F              
datatype     TMAX  TMIN  TOBS   TMAX   TMIN   TOBS
date                                              
2018-10-01   21.1   8.9  13.9  69.98  48.02  57.02
2018-10-02   23.9  13.9  17.2  75.02  57.02  62.96
2018-10-03   25.0  15.6  16.1  77.00  60.08  60.98
2018-10-04   22.8  11.7  11.7  73.04  53.06  53.06
2018-10-05   23.3  11.7  18.9  73.94  53.06  66.02

## Melting / Stacking

### Melting

In [11]:
wide_df = pd.read_csv('data/wide_data.csv')
wide_df.head()

,date,TMAX,TMIN,TOBS
0,2018-10-01,21.1,8.9,13.9
1,2018-10-02,23.9,13.9,17.2
2,2018-10-03,25.0,15.6,16.1
3,2018-10-04,22.8,11.7,11.7
4,2018-10-05,23.3,11.7,18.9


In [16]:
melted_df = wide_df.melt(
    id_vars='date',
    value_name='temp_C',
    var_name='Measurement'
)
melted_df.head()

,date,Measurement,temp_C
0,2018-10-01,TMAX,21.1
1,2018-10-02,TMAX,23.9
2,2018-10-03,TMAX,25.0
3,2018-10-04,TMAX,22.8
4,2018-10-05,TMAX,23.3


### Stacking

In [18]:
wide_df.set_index('date', inplace=True)
wide_df.head()

,TMAX,TMIN,TOBS
date,,,
2018-10-01,21.1,8.9,13.9
2018-10-02,23.9,13.9,17.2
2018-10-03,25.0,15.6,16.1
2018-10-04,22.8,11.7,11.7
2018-10-05,23.3,11.7,18.9


In [19]:
stacked_series = wide_df.stack()
stacked_series.head()

date            
2018-10-01  TMAX    21.1
            TMIN     8.9
            TOBS    13.9
2018-10-02  TMAX    23.9
            TMIN    13.9
dtype: float64

# Task 2

## Problematic data

In [20]:
df = pd.read_csv('data/dirty_data.csv')
df.head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False


In [21]:
df.describe()

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF
count,765.000000,577.000000,577.0,765.000000,765.000000,398.000000,11.000000
mean,5.360392,4.202773,NaN,2649.175294,-15.914379,8.632161,16.290909
std,10.002138,25.086077,NaN,2744.156281,24.242849,9.815054,9.489832
min,0.000000,0.000000,-inf,-11.700000,-40.000000,-16.100000,1.800000
25%,0.000000,0.000000,NaN,13.300000,-40.000000,0.150000,8.600000
50%,0.000000,0.000000,NaN,32.800000,-11.100000,8.300000,19.300000
75%,5.800000,0.000000,NaN,5505.000000,6.700000,18.300000,24.900000
max,61.700000,229.000000,inf,5505.000000,23.900000,26.100000,28.700000


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               765 non-null    object 
 1   station            765 non-null    object 
 2   PRCP               765 non-null    float64
 3   SNOW               577 non-null    float64
 4   SNWD               577 non-null    float64
 5   TMAX               765 non-null    float64
 6   TMIN               765 non-null    float64
 7   TOBS               398 non-null    float64
 8   WESF               11 non-null     float64
 9   inclement_weather  408 non-null    object 
dtypes: float64(7), object(3)
memory usage: 59.9+ KB


In [23]:
corrupted = df[
    df.SNOW.isna() | 
    df.SNWD.isna() | 
    df.PRCP.isna() | 
    df.TMAX.isna() | 
    df.TMIN.isna() | 
    df.TOBS.isna() | 
    df.WESF.isna()
    ]
corrupted.head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False


## Mitigating issues

In [24]:
# 1. make the date a datetime
df.date = pd.to_datetime(df.date)

# 2. save this information for later
station_qm_wesf = df[df.station == '?'].drop_duplicates('date').set_index('date').WESF

# 3. sort ? to the bottom
df.sort_values('station', ascending=False, inplace=True)

# 4. drop duplicates based on the date column keeping the first occurrence 
# which will be the valid station if it has data
df_deduped = df.drop_duplicates('date')

# 5. remove the station column because we are done with it
df_deduped = df_deduped.drop(columns='station').set_index('date').sort_index()

# 6. take valid station's WESF and fall back on station ? if it is null
df_deduped = df_deduped.assign(
    WESF=lambda x: x.WESF.combine_first(station_qm_wesf)
)

df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True


In [26]:
df_deduped.dropna().head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-30,0.0,0.0,-inf,6.7,-1.7,-0.6,1.8,False
2018-03-13,4.1,51.0,inf,5.6,-3.9,0.0,3.0,True
2018-03-21,0.0,0.0,-inf,2.8,-2.8,0.6,8.6,False
2018-04-02,9.1,127.0,inf,12.8,-1.1,-1.1,15.2,True


In [28]:
import numpy as np

df_deduped.assign(
    SNWD=lambda x: np.nan_to_num(x.SNWD)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-1.797693e+308,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-1.797693e+308,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-1.797693e+308,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,1.797693e+308,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,1.797693e+308,-4.4,-13.9,-13.9,NaN,True


In [29]:
df_deduped.assign(
    SNWD=lambda x: x.SNWD.clip(0, x.SNOW)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,0.0,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,0.0,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,0.0,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,229.0,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,127.0,-4.4,-13.9,-13.9,NaN,True


In [30]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(x.TMAX.median()),
    TMIN=lambda x: x.TMIN.fillna(x.TMIN.median()),
    # average of TMAX and TMIN
    TOBS=lambda x: x.TOBS.fillna((x.TMAX + x.TMIN) / 2)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,2732.5,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,2732.5,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True


In [31]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.clip(0, x.TMAX),
    TMIN=lambda x: x.TMIN.clip(0, x.TMIN),
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,0.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,0.0,0.0,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,0.0,0.0,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,0.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,0.0,0.0,-13.9,NaN,True


# Task 3

In [32]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04


In [33]:
weather = pd.read_csv('data/nyc_weather_2018.csv')
weather.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,",0.0


In [34]:
weather[weather.value > 0].head()

,date,datatype,station,attributes,value
8,2018-01-01T00:00:00,SNWD,GHCND:US1NJBG0018,",,N,",3.0
11,2018-01-01T00:00:00,SNWD,GHCND:US1NJBG0023,",,N,",25.0
40,2018-01-01T00:00:00,SNWD,GHCND:US1NJMN0048,",,N,",13.0
52,2018-01-01T00:00:00,SNWD,GHCND:US1NJMS0089,",,N,",13.0
56,2018-01-01T00:00:00,SNWD,GHCND:US1NJPS0005,",,N,",10.0


Find all entries where datatype is “PRCP“ and station ID containing “S0007”

In [40]:
weather[(weather.datatype=='PRCP') & (weather.station.str.contains("S0007"))]

,date,datatype,station,attributes,value
79,2018-01-01T00:00:00,PRCP,GHCND:US1NYNS0007,",,N,",0.0
311,2018-01-02T00:00:00,PRCP,GHCND:US1NYNS0007,",,N,",0.0
545,2018-01-03T00:00:00,PRCP,GHCND:US1NYNS0007,",,N,",0.0
788,2018-01-04T00:00:00,PRCP,GHCND:US1NYNS0007,",,N,",4.1
1044,2018-01-05T00:00:00,PRCP,GHCND:US1NYNS0007,",,N,",18.0
...,...,...,...,...,...
77762,2018-12-27T00:00:00,PRCP,GHCND:US1NYNS0007,",,N,",0.0
77974,2018-12-28T00:00:00,PRCP,GHCND:US1NYNS0007,",,N,",4.3
78191,2018-12-29T00:00:00,PRCP,GHCND:US1NYNS0007,",,N,",25.9
78408,2018-12-30T00:00:00,PRCP,GHCND:US1NYNS0007,",,N,",0.3


How many unique values in “attributes” variable?

In [49]:
weather.attributes.nunique()

26

What is the most frequent one?

In [51]:
weather.attributes.value_counts().head(1)

,,N,    34153
Name: attributes, dtype: int64